# [Assistants migration guide](https://platform.openai.com/docs/assistants/migration)

# Constants and Libraries

In [1]:
import os
from dotenv import load_dotenv # requires python-dotenv

if not load_dotenv("./../../config/credentials_my.env"):
    print("Environment variables not loaded, cell execution stopped")
else:
    print("Environment variables have been loaded ;-)")

QUESTION = "Create a visualization of a sinewave"

Environment variables have been loaded ;-)


# Create a client to connecto Azure OpenAI service and deployment

In [2]:
from openai import AzureOpenAI

# Create the client
client = AzureOpenAI(
    # api_key        = os.getenv("AZURE_OPENAI_API_KEY"),  
    # api_version    = os.getenv("AZURE_OPENAI_API_VERSION"), # at least 2024-02-15-preview
    # azure_endpoint = os.getenv("AZURE_OPENAI_ENDPOINT")
)
deployment_name = os.getenv("MODEL_DEPLOYMENT_NAME")
print(f"client.base_url: {client.base_url}.\nDeployment name: {deployment_name}.")

client.base_url: https://aiservicesiyva.openai.azure.com/openai/.
Deployment name: gpt-4.1.


# Check the files already uploaded, eventually delete them

In [3]:
def list_uploaded_files (client:AzureOpenAI, delete:bool=False):
    i = 0
    for file in client.files.list().data:
        i += 1
        if delete:
            print(f"File {i}: {file.filename} (id={file.id}) is being deleted...")
            client.files.delete(file.id) # un-comment this line if you want to delete it
        else:
            print(f"File {i}: {file.filename} (id={file.id})")

list_uploaded_files(client=client, delete=False) # here you may decide to delete **ALL** the files

# Upload the Assistant file(s)

In [4]:
from openai.types import FileObject
from pathlib import Path

DATA_FOLDER = "./data/"

def upload_file(client: AzureOpenAI, path: str) -> FileObject:
    with Path(path).open("rb") as f:
        return client.files.create(file=f, purpose="assistants")

arr = os.listdir(DATA_FOLDER)
assistant_files = []
for file in arr:
    filePath = DATA_FOLDER + file
    print(f"Uploading {filePath}...")
    assistant_files.append(upload_file(client, filePath))

file_ids = [file.id for file in assistant_files]

print(f"file_ids: {file_ids}")

list_uploaded_files(client=client, delete=False)

Uploading ./data/turbines.csv...
file_ids: ['assistant-LsS2rtKcMmQNi5FPNu1L8U']
File 1: turbines.csv (id=assistant-LsS2rtKcMmQNi5FPNu1L8U)


# Create an assistant with `code_interpreter` and `file_ids`

In [5]:
assistant = client.beta.assistants.create(
    name="Turbines Assistant",
    description="You are a helpful AI assistant who helps answering questions",
    instructions = """
        You are an assistant that can help manage wind turbine farm.
        The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.
        Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.
        """,
    model = deployment_name,
    tools = [{"type": "code_interpreter"}],
    tool_resources = {
        "code_interpreter": {"file_ids": file_ids}
    }
)

assistant

Assistant(id='asst_fuwPA0vvpmHmfR0u6XJNLOFd', created_at=1748874105, description='You are a helpful AI assistant who helps answering questions', instructions="\n        You are an assistant that can help manage wind turbine farm.\n        The turbines operating ranges are output voltages of 33kv-35kv and RPM of 15-25. Wind speed is measured in miles per hour.\n        Maintenance should occur every 12 months. Greet the user by saying, 'Welcome Turbine Management Assistant.\n        ", metadata={}, model='gpt-4.1', name='Turbines Assistant', object='assistant', tools=[CodeInterpreterTool(type='code_interpreter')], response_format='auto', temperature=1.0, tool_resources=ToolResources(code_interpreter=ToolResourcesCodeInterpreter(file_ids=['assistant-LsS2rtKcMmQNi5FPNu1L8U']), file_search=None), top_p=1.0)

# Create a conversation

## First, create a thread...
Note that `code_interpreter` and `file_search` are empty, because they are associated with the `assistant`, not to the `thread`

In [6]:
thread = client.beta.threads.create()
print(thread)

Thread(id='thread_ivifYUo7mTzi9kd2MM0pQ4WD', created_at=1748874106, metadata={}, object='thread', tool_resources=ToolResources(code_interpreter=None, file_search=None))


## ...then, add a message to the thread

In [7]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="What is the status of turbine 1001 and 1003"
)

thread_messages = client.beta.threads.messages.list(thread.id)

# print(thread_messages.model_dump_json(indent=2))

# Create a Run and check its status

In [8]:
import time, json

run              = client.beta.threads.runs.create(
    thread_id    = thread.id,
    assistant_id = assistant.id,
)

while client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status in ["queued", "in_progress"]:
    print(f"Run status: {run.status}")
    time.sleep(5)

print(f"Final run status: {client.beta.threads.runs.retrieve(thread_id=thread.id, run_id=run.id).status}\n\n")

# extract the result from the run, if successfull
for m in enumerate(reversed(client.beta.threads.messages.list(thread_id=thread.id).data),1):
    print(f">>> MESSAGE {m[0]}:\n{m[1].content[0].text.value}\n\n")

Run status: queued
Run status: queued
Final run status: completed


>>> MESSAGE 1:
What is the status of turbine 1001 and 1003


>>> MESSAGE 2:
Welcome Turbine Management Assistant.

I will check the uploaded file for the status of turbines 1001 and 1003. Let me review the data and provide you with the current status.


>>> MESSAGE 3:
Here is the current status for the turbines you requested:

- Turbine 1001:
  - Wind Speed: 32 mph
  - RPM: 18
  - Voltage: 33 kV
  - Last Maintenance Date: 2023-10-12

- Turbine 1003:
  - Wind Speed: 30 mph
  - RPM: 16
  - Voltage: 32 kV
  - Last Maintenance Date: 2023-11-30

Would you like a detailed assessment of their operational status against optimal ranges, or information about upcoming maintenance?




# Follow-up question below, then run the cell above to calculate and show it!

In [9]:
# Add a user question to the thread
message = client.beta.threads.messages.create(
    thread_id=thread.id,
    role="user",
    content="Show me the code you used to provide the previous answer"
)

# START teardown

In [10]:
# list & delete files in a single call

list_uploaded_files(client=client, delete=True)

File 1: turbines.csv (id=assistant-LsS2rtKcMmQNi5FPNu1L8U) is being deleted...


In [11]:
# delete messages in the current thread
for message in client.beta.threads.messages.list(thread.id):
    print (f"Deleting message id = <{message.id}>... of thread <{thread.id}>...")
    client.beta.threads.messages.delete(message_id=message.id, thread_id=thread.id)

# delete thread
print(f"\nDeleting thread {thread.id}...")
client.beta.threads.delete(thread_id=thread.id)

# delete assistant
print(f"\nDeleting assistant {assistant.id} ({assistant.name})...")
client.beta.assistants.delete(assistant.id)

Deleting message id = <msg_V9l7a88St5pDAM0gIDGoeHvm>... of thread <thread_ivifYUo7mTzi9kd2MM0pQ4WD>...
Deleting message id = <msg_xD8z3UogeL8zfJBiEgDIBp4Z>... of thread <thread_ivifYUo7mTzi9kd2MM0pQ4WD>...
Deleting message id = <msg_zWbEicH7R0rHcZvaQ6XLuAl3>... of thread <thread_ivifYUo7mTzi9kd2MM0pQ4WD>...
Deleting message id = <msg_GN4OPYGwNpYFcAabtPIjOQOS>... of thread <thread_ivifYUo7mTzi9kd2MM0pQ4WD>...

Deleting thread thread_ivifYUo7mTzi9kd2MM0pQ4WD...

Deleting assistant asst_fuwPA0vvpmHmfR0u6XJNLOFd (Turbines Assistant)...


AssistantDeleted(id='asst_fuwPA0vvpmHmfR0u6XJNLOFd', deleted=True, object='assistant.deleted')

# Teardown for *all** assistants and messages [(but **NOT** threads!)](https://learn.microsoft.com/en-us/answers/questions/2153170/assistants-api-where-and-how-long-are-entities-sav)
When creating a new thread with the Assistants API, thereby creating a stateful entity of: Threads, messages, where is this data stored, can I access the resource that stores these? Or is the resource managed entirely by Microsoft, and inaccessible to me?
When you create a new thread with the Assistants API, the data (threads, messages, etc.) is stored in a secure, Microsoft-managed storage account. This storage is logically separated to ensure data security. As a user, you do not have direct access to the underlying storage resources. Instead, you interact with the data through the API endpoints provided by Microsoft.

If a thread with messages is created via the API, and the ID is lost, is there then no route to access and delete this thread? As in, would there be a way to somehow fetch all threads related to a specific OpenAI Azure resource?
If you lose the thread ID, there is no direct way to retrieve or delete the thread through the API. The Assistants API does not currently provide a method to list all threads associated with a specific OpenAI Azure resource. Therefore, it's crucial to manage and store thread IDs securely within your application to ensure you can access and manage them as needed.

Is stateful entities stored INDEFINITELY unless deleted? Or is the a time to live if not used?
All used data persists in this system unless you explicitly delete this data. Use the delete function with the thread ID of the thread you want to delete. Clearing the Run in the Assistants Playground does not delete threads, however deleting them using delete function will not list them in the thread page.

In [12]:
from datetime import datetime

assistants = client.beta.assistants.list(limit=100).data
i=0
while len(assistants) > 0:
    for assistant in assistants: 
        i=i+1
        print(f"Deleting assistant {i}: {assistant.id} ({assistant.name}) created at {datetime.fromtimestamp(assistant.created_at).strftime('%Y-%m-%d %H:%M:%S')}...")
        client.beta.assistants.delete(assistant.id)
    assistants = client.beta.assistants.list(limit=100).data

print (f"\n{i} assistants have been successfully deleted.")


0 assistants have been successfully deleted.
